<a href="https://colab.research.google.com/github/lukemerrick/image-text-embedding/blob/main/initial_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>